# Main class for running the molecule environment

In [1]:
import gym
from gym import wrappers, logger
from MoleculeAgent import MoleculeAgent
from rdkit import Chem
# TODO: prompt user when wrong goal entered
# import chem, make a mol object valid and only move from there.
# Add pop in modifications

In [16]:
def validate(mol):
        try:
            molecule = Chem.MolFromSmiles(mol)
            smiles = Chem.MolToSmiles(molecule, isomericSmiles=True)
            mol2 = Chem.MolFromSmiles(smiles)
        except:
            print("Entered input is chemically invalid")
            return False
        else:
            
            return True


def main():   
    
    menu = input("1: Set Environment initial molecule and optimization goal. \n2: Use a Random set and initial moecule. \n")
    if menu == "2":
        mol = 1
        goal = 1
    elif menu == "1":
        # Get valid input goal and start state from user
        validgoal = False
        validmol = False
        while validgoal == False:
            goal = input("Enter a molecule to act as an optimisation Goal. \n ")
            validgoal = validate(goal)  

        while validmol == False:
            mol = input("Enter an atom or molecule to act as the start state of the environment: ")
            validmol = validate(mol)
            
    env = gym.make("gym_molecule:molecule-v0", mol = mol, goal= goal)
    print("Environment initialiased.")
    #Create a new Molecule Agent object. 
    agent = MoleculeAgent(env.action_space)
    
    
    episode_count = 3
    reward = 0
    done = False

    #The 'RL algorithm' learns for 2 episodes of undefined length.
    for i in range(1, episode_count +1 ):
        print("EPISODE:", i)
        
        #The env resets.
        ob = env.reset()
        print("Env original state: ", ob)
        
        #While what is True? Maybe While done is False?
        for x in range(20):
            
            #The agent chooses its action based on the tuple from the current env state.
            action = agent.act(ob, reward, done)
            print()
            print("Agent's chosen action: ", action) 
            
            #The new tuple is returned because the env has considered the agent's action.
            ob, reward, done, _ = env.step(action)
            
            
            print("Env new state:", ob, "Reward to agent:", reward, "Done:", done,"Info: ", _)
            
            #The env renders. 
            env.render()
            
            if done == True:
                print("Molecule Optimized")
                break

            
        
            #If the env reaches a terminating state, the current iteration ends and the next 
            #episode begins. 

        
    env.close()
    
    
if __name__ == '__main__':
    main()

1: Set Environment initial molecule and optimization goal. 
2: Use a Random set and initial moecule. 
1
Enter a molecule to act as an optimisation Goal. 
 CCC
Enter an atom or molecule to act as the start state of the environment: C
Environment initialiased.
EPISODE: 1
Env original state:  None

Agent's chosen action:  4
add bracketed atom
['bracketed atom']
calculating Reward
Env new state: 0.0 Reward to agent: 1 Done: False Info:  {}

Agent's chosen action:  1
['bracketed atom', 'C']
calculating Reward
Env new state: 50.0 Reward to agent: 1 Done: False Info:  {}

Agent's chosen action:  0
['bracketed atom', 'C', 'C']
Env new state: 100.0 Reward to agent: 0 Done: True Info:  {}
Molecule Optimized
EPISODE: 2
Env original state:  None

Agent's chosen action:  0
['C']
calculating Reward
Env new state: 50.0 Reward to agent: 1 Done: False Info:  {}

Agent's chosen action:  3
['C', 'C']
Env new state: 100.0 Reward to agent: 0 Done: True Info:  {}
Molecule Optimized
EPISODE: 3
Env original s